# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

In [2]:
# The business objectives of this activity is to identify what features of the dataset are best used to predict and
# determine used car prices.  Our inventory of resoruces include various data sets, but for this exercise we will use
# a subset of 426k vehicles of a 3M data set.  The goals and success criteria for this exercise are to determine
# which features are drivers of used car prices.  We will use various data modeling techniques within Sklearn to
# build and test appropiate models.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
# We have the initial dataset so we'll first look through the columns and how the data is constructed for the 429k venicles.
# From there we will document the description of data in a data description report, provide findings of exploration
# in a data exploration report and assess overall quality of data by looking for gaps, inconsistencies, common erros
# etc. and document them in a data quality report.

### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [66]:
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from warnings import filterwarnings 
filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import mean_squared_error
import warnings

from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display="diagram")

from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display="diagram")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [74]:
df = pd.read_csv('data/vehicles.csv', index_col=0)

In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 426880 entries, 7222695916 to 7301591129
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region        426880 non-null  object 
 1   price         426880 non-null  int64  
 2   year          425675 non-null  float64
 3   manufacturer  409234 non-null  object 
 4   model         421603 non-null  object 
 5   condition     252776 non-null  object 
 6   cylinders     249202 non-null  object 
 7   fuel          423867 non-null  object 
 8   odometer      422480 non-null  float64
 9   title_status  418638 non-null  object 
 10  transmission  424324 non-null  object 
 11  VIN           265838 non-null  object 
 12  drive         296313 non-null  object 
 13  size          120519 non-null  object 
 14  type          334022 non-null  object 
 15  paint_color   296677 non-null  object 
 16  state         426880 non-null  object 
dtypes: float64(2), int64(1), object(14)

In [76]:
df.head(5)

,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
id,,,,,,,,,,,,,,,,,
7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [77]:
df.model.value_counts()

f-150                      8009
silverado 1500             5140
1500                       4211
camry                      3135
silverado                  3023
                           ... 
plymouth fury 1               1
f550 box truck delivery       1
duramax 2500hd                1
3 s sport                     1
Paige Glenbrook Touring       1
Name: model, Length: 29649, dtype: int64

In [78]:
# in looking at the values of various columns, we're going to remove some that I feel will not significantly impact
#the price. These include region as we can sell nationwide, title_status, VIN, drive, state, manufacturer since its
# really a higher altitude than model
df.drop(columns=['region', 'title_status', 'VIN', 'drive', 'state', 'manufacturer'], inplace=True)

In [79]:
#I will then drop NAs. Note I do this after removing the columns so it doesn't drop too many rows. We are left with 85,927 rows
df = df.dropna()
print(len(df))

85927


In [80]:
#Remove outliers
df = df[(df['price'] <= 99999) & (df['price'] >= 1000)]  
print(len(df))

79964


In [81]:
#The data set is WAY too large and keep crashing - I'm limiting to 1000 rows
df = df.iloc[:1000]
df.head()

,price,year,model,condition,cylinders,fuel,odometer,transmission,size,type,paint_color
id,,,,,,,,,,,
7316356412,15000,2013.0,f-150 xlt,excellent,6 cylinders,gas,128000.0,automatic,full-size,truck,black
7314560853,19900,2004.0,f250 super duty,good,8 cylinders,diesel,88000.0,automatic,full-size,pickup,blue
7313406529,14000,2012.0,odyssey,excellent,6 cylinders,gas,95000.0,automatic,full-size,mini-van,silver
7312847466,22500,2001.0,f450,good,8 cylinders,diesel,144700.0,manual,full-size,truck,white
7312144944,15000,2017.0,charger rt 4dr sedan,excellent,8 cylinders,gas,90000.0,automatic,mid-size,sedan,grey


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [84]:
#Need to get dummies for object columns
df = pd.get_dummies(df)
df.head()

,price,year,odometer,model_124 spider,model_1500,model_1500 avalanche ltz 4x4,model_1500 big horn,model_1500 express,model_1500 slt,model_1500 yukon slt,...,paint_color_blue,paint_color_brown,paint_color_custom,paint_color_green,paint_color_grey,paint_color_orange,paint_color_red,paint_color_silver,paint_color_white,paint_color_yellow
id,,,,,,,,,,,,,,,,,,,,,
7316356412,15000,2013.0,128000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7314560853,19900,2004.0,88000.0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7313406529,14000,2012.0,95000.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7312847466,22500,2001.0,144700.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7312144944,15000,2017.0,90000.0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [85]:
X = pd.get_dummies(df.drop('price', axis = 1))
y = df['price']
all_features_linreg = LinearRegression(fit_intercept=False).fit(X, y)
linreg_mse = mean_squared_error(all_features_linreg.predict(X), y)

print(all_features_linreg)
print(all_features_linreg.coef_)
print(linreg_mse)

LinearRegression(fit_intercept=False)
[ 7.28873739e+02 -4.00987758e-02 -1.10548802e+09 -1.29971814e+10
 -1.29971823e+10 -1.29971710e+10 -1.29971831e+10 -1.29971751e+10
 -1.10546323e+09 -1.10545637e+09 -1.29971570e+10 -1.10544390e+09
 -1.10547143e+09 -1.10547385e+09 -1.10548052e+09 -1.10549175e+09
 -1.10549416e+09 -1.29971665e+10 -1.10547562e+09 -1.10547575e+09
 -1.10547420e+09 -1.10542267e+09 -6.47870762e+10 -1.29971719e+10
 -1.29970910e+10 -1.29971711e+10 -1.10547376e+09 -1.10547213e+09
 -1.10547464e+09 -1.10546920e+09 -1.10547779e+09 -1.10547153e+09
 -1.10547164e+09 -1.29971766e+10 -1.29971661e+10 -1.29971766e+10
 -1.10540401e+09 -1.10547132e+09 -1.29971596e+10 -1.10546862e+09
 -1.10546550e+09 -1.10547094e+09 -1.10547161e+09 -1.10545415e+09
 -1.29971825e+10 -1.10546889e+09 -1.10549330e+09 -1.10548660e+09
 -1.10546346e+09 -1.29971584e+10 -1.10547014e+09 -1.10546646e+09
 -1.10546391e+09 -1.29971679e+10 -1.29971554e+10 -1.10548367e+09
 -1.29971355e+10 -1.29971662e+10 -1.29971608e+10 -1.

In [87]:
#Create Train and Test

X_train, X_test, y_train, y_test = train_test_split(df.drop('price', axis = 1), np.log1p(df.price), 
                                                   random_state=42, test_size = 0.3)
print(X_train.shape)
print(X_test.shape)

(700, 600)
(300, 600)


In [88]:
#Feature Selection Pipeline
selector_pipe = Pipeline([('selector', SequentialFeatureSelector(LinearRegression())),
                         ('model', LinearRegression())])
selector_pipe

param_dict = {'selector__n_features_to_select': [2, 3, 4, 5]}
selector_grid = GridSearchCV(selector_pipe, param_grid=param_dict)
selector_grid.fit(X_train, y_train)
train_preds = selector_grid.predict(X_train)
test_preds = selector_grid.predict(X_test)
selector_train_mse = mean_squared_error(y_train, train_preds)
selector_test_mse = mean_squared_error(y_test, test_preds)

In [89]:
#Check
print(f'Train MSE: {selector_train_mse}')
print(f'Test MSE: {selector_test_mse}')

Train MSE: 0.4067914716751987
Test MSE: 0.4790589486222215


In [90]:
#Ridge Selection
ridge_param_dict = {'ridge__alpha': np.logspace(0, 10, 50)}
ridge_pipe = Pipeline([('scaler', StandardScaler()), 
                      ('ridge', Ridge())])
ridge_grid = GridSearchCV(ridge_pipe, param_grid=ridge_param_dict)
ridge_grid.fit(X_train, y_train)
ridge_train_preds = ridge_grid.predict(X_train)
ridge_test_preds = ridge_grid.predict(X_test)
ridge_train_mse = mean_squared_error(y_train, ridge_train_preds)
ridge_test_mse = mean_squared_error(y_test, ridge_test_preds)
print(f'Train MSE: {ridge_train_mse}')
print(f'Test MSE: {ridge_test_mse}')

Train MSE: 0.06282367963798448
Test MSE: 0.3500135289787307


In [91]:
best_estimator = selector_grid.best_estimator_
best_selector = best_estimator.named_steps['selector']
best_model = selector_grid.best_estimator_.named_steps['model']
feature_names = X_train.columns[best_selector.get_support()]
coefs = best_model.coef_

In [92]:
print(best_estimator)
print(f'Features from best selector: {feature_names}.')
print('Coefficient values: ')
print('===================')
pd.DataFrame([coefs.T], columns = feature_names, index = ['model'])

Pipeline(steps=[('selector',
                 SequentialFeatureSelector(estimator=LinearRegression(),
                                           n_features_to_select=5)),
                ('model', LinearRegression())])
Features from best selector: Index(['year', 'condition_fair', 'condition_like new', 'type_sedan',
       'type_truck'],
      dtype='object').
Coefficient values: 


,year,condition_fair,condition_like new,type_sedan,type_truck
model,0.014502,-1.162304,0.422705,-0.458129,0.560681


### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [94]:
# As I went through the data and tested various models, I see the Linear Regression model selected 5 features
# that best predicted price.  However these are really 3 features which includes year, condition and type of vehicle.
# I would say that due to system limitations I had to select a subset of data and if I was able to analyze the full
# 3M vehciles, we may come up with different information.  Perhaps we can find a quantum computer to do that!

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.

In [ ]:
# I understand it's important that you're able to predict the sales price of used vehicles to ensure you not only
# offer the right purchase price from sellers, but also list vehicles at a competitive rate while maximizing
# your profits.  Based on our analysis, we found that there are 3 major catagories that best contribute to price
# prediction.  These include the year of the vehicle, condition of the vehicle and type.  When the condition is fair 
# or like new, we will have the best predictive capabilities.  When the type is a sedan or a truck, we will have
# the best modeling as well.